# load the libraries

In [2]:
import pandas as pd
import datetime
import numpy as np
import cPickle as pickle
from gensim import corpora, models, matutils
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import defaultdict
from lda2vec import preprocess, Corpus
from topicModelingClass import topicModeling

In [3]:
with open('master_total_df.p','rb') as f:
    master_total_df = pickle.load(f)
master_total_df.head(2)

,index,company,jobdesc,city,state,title,sourcesite,views,days_posted,post_start_date,link,base_title,parsed_title,parsed_title_i,expanded_title,prefix_title
0,0,Advocate Health Care,"\nAs part of Advocate Health Care, Advocate Ch...",Oak Lawn,IL,Clinical Practice Specialist - 4 Hope,ind,0,NaN,NaN,http://www.indeed.com/viewjob?jk=6244f7f3a4861...,specialist,"[clinical, practice, specialist, , , 4, hope]",[2],practice specialist,practice
1,1,University of Washington Medical Center,\nThe University of Washington (UW) is proud t...,Seattle,WA,WEB DEVELOPER,ind,0,NaN,NaN,http://www.indeed.com/viewjob?jk=3d6a9f18f5301...,developer,"[web, developer]",[1],web developer,web


In [4]:
alltext = master_total_df['jobdesc'].values
def uniuncode(x):
    try:
        return unicode(x.decode('utf-8')).replace('\n',' -').lower()

    except:
        try:
            return unicode(x).replace('\n',' -').lower()
        except:
            print x
            return x 
sometext = [uniuncode(x) for x in alltext[:100]]


In [6]:
# max words grabbed per document
max_words = 10000

start = datetime.datetime.now()

# convert text to unicode (if not already)
# in my case text is already in unicode
# tokenize uses spacy under the hood
tokens, vocab = preprocess.tokenize(sometext, max_words, merge=False, n_threads=4)

print '1. tokens made'
# he made a generic corpus based on default dictionary
# see documentation
corpus = Corpus()

# Make a ranked list of rare vs frequent words
corpus.update_word_count(tokens)
corpus.finalize()

print '2. corpus updated'

# The tokenization uses spaCy indices, and so may have gaps
# between indices for words that aren't present in our dataset.
# This builds a new compact index
compact = corpus.to_compact(tokens)

print '3. corpus compacted'


# Remove extremely rare words
pruned = corpus.filter_count(compact, min_count=10)

# Convert the compactified arrays into bag of words arrays
bow = corpus.compact_to_bow(pruned)

print '4. bag of words (BOW) made'

# Words tend to have power law frequency, so selectively
# downsample the most prevalent words
clean = corpus.subsample_frequent(pruned)
doc_ids = np.arange(pruned.shape[0])
flattened, (doc_ids,) = corpus.compact_to_flat(pruned, doc_ids)
assert flattened.min() >= 0

print '5. corpus flattened'

# Fill in the pretrained word vectors
n_dim = 300
fn_wordvc = '../08-wordvectors-pretrained/GoogleNews-vectors-negative300.bin'
vectors, s, f = corpus.compact_word_vectors(vocab, filename=fn_wordvc)

print '6. pretrained words loaded'

# Save all of the preprocessed files
pickle.dump(vocab, open('vocab.pkl', 'w'))
pickle.dump(corpus, open('corpus.pkl', 'w'))
np.save("flattened", flattened)
np.save("doc_ids", doc_ids)
np.save("pruned", pruned)
np.save("bow", bow)
np.save("vectors", vectors)

print '7. files saved'



1. tokens made
2. corpus updated
3. corpus compacted
4. bag of words (BOW) made
5. corpus flattened
2 <SKIP>  -->  SKIP
3 ,  -->  上
5 -  -->  上
6 .  -->  上
15 :  -->  上
24 )  -->  上
26 (  -->  上
35 -•  -->  -4
36   -->  上
40 ’s  -->  qs
50 ;  -->  上
79 -experience  -->  experience
82 -the  -->  dthe
112 /  -->  上
115 -ability  -->  ability
133 -strong  -->  strong
143 and/or  -->  andor
151 ?  -->  上
162 -30+  -->  -0
203 –  -->  上
245 -we  -->  iwe
257 -work  -->  work
263 -job  -->  job
266 --  -->  -4
295 !  -->  上
303 -a  -->  qa
304 -excellent  -->  excellent
307 ’  -->  上
321 you’ll  -->  youll
327 -this  -->  Tthis
334 -you  -->  Ryou
354 -bachelor  -->  bachelor
368 -what  -->  whatā
383 '  -->  上
386 -qualifications  -->  qualifications
387 -responsibilities  -->  responsibilities
393 -about  -->  about
394 -at  -->  qat
456 -as  -->  Das
463 "  -->  上
492 -  -->  -4
497 -develop  -->  develop
502 -knowledge  -->  aknowledge
523 -working  -->  working
526 -our  -->  Touré
546

KeyboardInterrupt: 